# Imports

In [1]:
import re
import sys
sys.path.append("../../")
import pandas as pd
from transformers import pipeline
from bechdelai.data.scenariotheque import get_scripts, get_file_content
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from textsplit.tools import SimpleSentenceTokenizer
from textsplit.tools import get_penalty, get_segments
from textsplit.algorithm import split_optimal, split_greedy, get_total

In [2]:
scripts = get_scripts()

In [14]:
movie = "Antoinette dans les Cévennes"
url = scripts.loc[scripts["Titre"] == movie, "PDF"].values[0]

In [15]:
script = get_file_content(url)

In [16]:
# D'après https://www.studiobinder.com/blog/how-to-write-a-screenplay/
RE_SCENE = r"(?<=INT. |EXT. ).*(?=)"
RE_CHARACTER = r"([A-ZÏ]+)\s"

In [17]:
content = script.replace("\n\n", "\n")

In [20]:
parsed = {}
scene_count = -1
scene_index = -1
character_index = -1
character_name = ""
for idx, line in enumerate(content.split("\n")[35:120]):
    scene = re.search(RE_SCENE, line)
    if scene:
        scene_count += 1
        parsed[scene_count]= {"location": scene.group(0)}
        scene_index = idx
    character = re.search(RE_CHARACTER, line)
    if character:
        character_index = idx
        character_name = character.group(0).strip()
        parsed[scene_count]["dialogs"] = {}
        
    if idx != 0 and idx >= scene_index + 1 and idx > character_index:
        parsed[scene_count]["action"] = line.strip() if not parsed[scene_count].get("action") else parsed[scene_count]["action"] + " " + line.strip()
    if idx != 0 and idx >= character_index + 1 and idx > scene_index:
        parsed[scene_count]["dialogs"][character_name] = line.strip() if not parsed[scene_count]["dialogs"].get(character_name) else parsed[scene_count]["dialogs"][character_name] + " " + line.strip()
print(parsed)

{}


In [175]:
re.findall(RE_SCENE, content)

['NUIT. CHAMBRE/ APPARTEMENT PARISIEN.',
 'NUIT. ROUTE DE PROVINCE / CAMION.',
 'NUIT. CAMION DE YANN.',
 'JOUR. CUISINE AMERICAINE/ APPARTEMENT RAF',
 'JOUR. RUE SAINT-HONORE.',
 'JOUR. RUE DE PARIS.',
 'NUIT. HOPITAL/ LOCAL INFIRMIERES/',
 'NUIT. HOPITAL/ ENTREE DES URGENCES.',
 'NUIT. SALLE DE TRI/ URGENCES.',
 'NUIT. 2EME SALLE DE TRI/ URGENCES ',
 'NUIT. SALLE DES TRANSMISSIONS/ URGENCES',
 'NUIT. SALLE D’ATTENTE/ URGENCES',
 'RAF',
 'NUIT. BOX VIEILLE DAME/ AUTRE BOX',
 'NUIT. SALLE D’ATTENTE.',
 'NUIT. LAVABOS-COULOIRS.',
 'NUIT. SALLE D’ATTENTE.',
 'NUIT. BOX DES INFIRMIERES.',
 'NUIT. COULOIR DES BOX. ',
 'NUIT. BOX YANN.',
 'NUIT. BOX DES INFIRMIERES / COULOIR DES',
 'NUIT. SALLE D’ATTENTE.',
 'NUIT. BOX DE RAF. ',
 'NUIT/ COULOIRS BOX.',
 'NUIT. BOX YANN.',
 'NUIT. BOX DE RAF.',
 'NUIT. COULOIRS DES BOX.',
 'NUIT. COULOIRS RADIO.',
 'NUIT. COUR DES URGENCES.',
 'NUIT. COULOIR. RADIO.',
 'NUIT. PIECE RADIO.',
 'NUIT. BOX DE RAF.',
 'NUIT. BOX DE RAF/ BOX DE YANN.',
 'NUIT. DE

In [135]:
re.findall(RE_CHARACTER, content)

['L ',
 'F ',
 'R ',
 'A ',
 'C ',
 'T ',
 'U ',
 'R ',
 'CHAZ ',
 'APPARTEMENT ',
 'DIX ',
 'ROUTE ',
 'DE ',
 'CAMION ',
 'DE ',
 'CUISINE ',
 'APPARTEMENT ',
 'ET ',
 'LA ',
 'BOUCHE ',
 'A ',
 'RUE ',
 'CRS ',
 'A ',
 'CRS ',
 'CRS ',
 'CRS ',
 'CRS ',
 'LE ',
 'CRS ',
 'CRS ',
 'CRS ',
 'RUE ',
 'DE ',
 'GENTIMENT ',
 'LUI ',
 'TEND ',
 'UN ',
 'LOCAL ',
 'FERMANT ',
 'SON ',
 'ENTREE ',
 'DES ',
 'LE ',
 'POMPIER ',
 'LE ',
 'POMPIER ',
 'AU ',
 'LE ',
 'POMPIER ',
 'LE ',
 'POMPIER ',
 'LE ',
 'POMPIER ',
 'LE ',
 'POMPIER ',
 'SALLE ',
 'DE ',
 'NE ',
 'RELEVE ',
 'LE ',
 'EME ',
 'SALLE ',
 'DE ',
 'SALLE ',
 'DES ',
 'INT ',
 'INFIRMIERE ',
 'VOYANT ',
 'BRACELET ',
 'DE ',
 'INFIRMIERE ',
 'INFIRMIERE ',
 'INFIRMIERE ',
 'SALLE ',
 'TEXTO ',
 'BOX ',
 'VIEILLE ',
 'AUTRE ',
 'AU ',
 'LA ',
 'VIEILLE ',
 'A ',
 'LA ',
 'VIEILLE ',
 'LA ',
 'VIEILLE ',
 'SALLE ',
 'EN ',
 'REVIENT ',
 'CALME ',
 'ET ',
 'LA ',
 'A ',
 'SALLE ',
 'A ',
 'A ',
 'AU ',
 'UN ',
 'DOSSIER ',
 'A ',